In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

# Load data
train = pd.read_csv("data/train.csv")
features = pd.read_csv("data/features.csv")
stores = pd.read_csv("data/stores.csv")

df = (
    train
    .merge(features, on=["Store", "Date", "IsHoliday"])
    .merge(stores, on="Store")
)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Dept", "Date"])
df.head()

In [ ]:
store = 1
dept = 1

ts = (
    df[(df["Store"] == store) & (df["Dept"] == dept)]
    [["Date", "Weekly_Sales"]]
    .rename(columns={"Date": "ds", "Weekly_Sales": "y"})
    .sort_values("ds")
)
ts.head()


In [ ]:
model = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode="additive"
)

model.fit(ts)


In [ ]:
future = model.make_future_dataframe(periods=12, freq="W")
forecast = model.predict(future)
forecast.tail()


In [ ]:
fig1 = model.plot(forecast)
plt.title(f"12-Week Forecast — Store {store}, Dept {dept}")
plt.tight_layout()
plt.savefig("images/forecast_store1_dept1.png")
plt.show()


In [ ]:
fig2 = model.plot_components(forecast)
plt.tight_layout()
plt.savefig("images/components_store1_dept1.png")
plt.show()


In [ ]:
Insights
• Weekly seasonality is strong, consistent with cyclical shopping behavior in large-format retail.
• Holiday periods show visible uplift relative to baseline, highlighting the impact of seasonal events.
• This baseline forecast will be used as an interpretable benchmark against more complex global models (e.g., LightGBM) later in the system.